In [1]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

path = r"C:\Users\patri\data-analyst-interview\data\artists.db"

In [2]:
conn = sqlite3.connect(path)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,artists,artists,2,"CREATE TABLE ""artists"" (\n""Artist ID"" INTEGER,..."
1,table,artworks,artworks,159,"CREATE TABLE ""artworks"" (\n""Artwork ID"" INTEGE..."


In [3]:
artists = pd.read_sql("""SELECT *
                        FROM artists;""", conn)
artists["Artist ID"] = artists["Artist ID"].astype("str")

In [4]:
artwork = pd.read_sql("""SELECT *
                        FROM artworks;""", conn)
artwork["Artist ID"] = artwork["Artist ID"].astype("str")
artwork["Artwork ID"] = artwork["Artwork ID"].astype("str")

# Question 1

In [5]:
pd.read_sql("""SELECT Name, MAX(`Death Year` - `Birth Year`)
                        FROM artists;""", conn)

,Name,MAX(`Death Year` - `Birth Year`)
0,"Union Paper Bag Machine Company, Philadelphia, PA",130


#### That isn't a real artist, so here is the top 5, so we can get some actual names

In [6]:
pd.read_sql("""SELECT Name, `Death Year` - `Birth Year`
                        FROM artists
                        ORDER BY `Death Year` - `Birth Year` DESC
                        LIMIT 5;""", conn)

,Name,`Death Year` - `Birth Year`
0,"Union Paper Bag Machine Company, Philadelphia, PA",130
1,Leni Matthaei,108
2,B. Efimov,108
3,Manoel de Oliveira,107
4,Hans Erni,106


#### Leni Matthaei has made the most artworks on display at MoMA

# Question 2

In [7]:
pd.read_sql("""SELECT artists.Name, COUNT(1) AS Frequency
                        FROM artists
                        INNER JOIN artworks ON artists.`Artist ID` = artworks.`Artist ID`
                        GROUP BY 1
                        ORDER BY 2 DESC
                        LIMIT 10;""", conn)

,Name,Frequency
0,Eugène Atget,5050
1,Louise Bourgeois,3318
2,Ludwig Mies van der Rohe,2566
3,Unknown photographer,1575
4,Jean Dubuffet,1435
5,Lee Friedlander,1317
6,Pablo Picasso,1310
7,Marc Chagall,1162
8,Henri Matisse,1063
9,Pierre Bonnard,894


# Question 3

Here I think it's a good idea to not only check for dimmensions of a rectangle area but also the circle's surface area, so using the formula

$ A = L\cdot w + \pi \cdot r^2 $

In [8]:
pd.read_sql("""SELECT artists.Name, SUM(`Height (cm)`*`Width (cm)`) + SUM((`Diameter (cm)`/2)*(`Diameter (cm)`/2)*3.14) AS `Surface Area`
                        FROM artists
                        INNER JOIN artworks ON artists.`Artist ID` = artworks.`Artist ID`
                        GROUP BY 1
                        ORDER BY 2 DESC
                        LIMIT 10""", conn)

,Name,Surface Area
0,James Lee Byars,1.040749e+07
1,Ludwig Mies van der Rohe,7.582402e+06
2,Louise Bourgeois,4.441873e+06
3,Frank Lloyd Wright,2.967754e+06
4,Robert Rauschenberg,2.770040e+06
5,Henri Matisse,1.736799e+06
6,Jean Dubuffet,1.563943e+06
7,Alexander Calder,1.556715e+06
8,Frank Stella,1.329153e+06
9,Kiki Smith,9.676348e+05


# Question 4

In [9]:
innermerge = pd.merge(artists, artwork, on='Artist ID', how='inner')
innermerge.head()

,Artist ID,Name_x,Nationality,Gender,Birth Year,Death Year,Artwork ID,Title,Name_y,Date,...,Classification,Object Number,Diameter (cm),Circumference (cm),Height (cm),Length (cm),Width (cm),Depth (cm),Weight (kg),Duration (s)
0,1,Robert Arneson,American,Male,1930.0,1992.0,33599,Study for Head Bath,Robert Arneson,1977,...,Drawing,67.1981,NaN,NaN,105.7,NaN,75.9,NaN,NaN,NaN
1,1,Robert Arneson,American,Male,1930.0,1992.0,64139,General Nuke,Robert Arneson,1986,...,Print,192.1997,NaN,NaN,80.5,NaN,59.2,NaN,NaN,NaN
2,2,Doroteo Arnaiz,Spanish,Male,1936.0,NaN,61629,BAS-RELIEF,Doroteo Arnaiz,None,...,Print,87.1965,NaN,NaN,54.7,NaN,39.7,NaN,NaN,NaN
3,3,Bill Arnold,American,Male,1941.0,NaN,45972,Honey under Sink,Bill Arnold,1971,...,Photograph,87.1972,NaN,NaN,38.1,NaN,58.2,NaN,NaN,NaN
4,3,Bill Arnold,American,Male,1941.0,NaN,45997,Honey under Chair,Bill Arnold,1971,...,Photograph,88.1972,NaN,NaN,38.1,NaN,57.8,NaN,NaN,NaN


In [10]:
#We slice it to only get year
innermerge["AcquiYear"] = innermerge["Acquisition Date"].str.slice(0,4)
innermerge["AcquiYear"] = innermerge["AcquiYear"].astype("float")
lifetime_sellers = innermerge.loc[(innermerge["AcquiYear"] < innermerge["Death Year"]) &
                                  (innermerge["AcquiYear"] > innermerge["Birth Year"]), "Name_x"].unique()
lifetime_sellers

array(['Robert Arneson', 'Jean (Hans) Arp', 'J. Arrelano Fischer', ...,
       'Munetsugu Satomi', 'Ambrogio Pozzi', 'Egidio Bonfante'],
      dtype=object)

In [11]:
len(lifetime_sellers)

2538

#### MoMA acquired artwork from 2538 unique artists in their lifetime

# Question 5

In [12]:
artists.describe()

,Birth Year,Death Year
count,11237.000000,4579.000000
mean,1930.852719,1974.287399
std,34.531997,31.153665
min,1730.000000,1795.000000
25%,1910.000000,1958.000000
50%,1936.000000,1980.000000
75%,1956.000000,1998.000000
max,2012.000000,2017.000000


#### About 3000 missing nationalities
#### About 3000 missing genders
#### About 4000 missing birth years
#### About 10000 missing death years, probably because not dead yet

In [13]:
artists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15091 entries, 0 to 15090
Data columns (total 6 columns):
Artist ID      15091 non-null object
Name           15091 non-null object
Nationality    12603 non-null object
Gender         12019 non-null object
Birth Year     11237 non-null float64
Death Year     4579 non-null float64
dtypes: float64(2), object(4)
memory usage: 707.5+ KB


In [14]:
artists["Artist ID"].str.isnumeric().value_counts()

True    15091
Name: Artist ID, dtype: int64

#### Good news is there's no repeating id

In [15]:
len(artists["Artist ID"].unique())

15091

#### 255x Nationality unknown instead of NaN
#### 2x Nationality Unknown instead of NaN
#### 2x nationality unknown instead of NaN
#### 2x Various, not specific

In [16]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(artists["Nationality"].value_counts())

American               5198
German                  930
French                  839
British                 835
Italian                 531
Japanese                498
Swiss                   280
Dutch                   265
Nationality unknown     255
Austrian                243
Canadian                196
Russian                 188
Brazilian               155
Spanish                 153
Argentine               139
Swedish                 130
Mexican                 128
Polish                  125
Danish                  119
Belgian                  89
Czech                    83
Chinese                  81
Israeli                  75
South African            69
Chilean                  61
Finnish                  60
Cuban                    58
Australian               56
Hungarian                52
Norwegian                49
Colombian                42
Venezuelan               41
Korean                   35
Peruvian                 34
Scottish                 28
Indian              

#### 6 of them are written as male instead of Male

In [17]:
artists["Gender"].value_counts()

Male      9820
Female    2193
male         6
Name: Gender, dtype: int64

In [18]:
artwork.describe()

,Diameter (cm),Circumference (cm),Height (cm),Length (cm),Width (cm),Depth (cm),Weight (kg),Duration (s)
count,1399.000000,10.000000,111893.000000,736.000000,111003.000000,11443.000000,298.000000,3.084000e+03
mean,23.248939,44.868020,37.712992,89.117417,38.176838,18.291359,1248.278691,7.830060e+03
std,45.460079,28.631604,48.151347,329.717487,67.250118,57.703925,11856.456824,1.185046e+05
min,0.635000,9.900000,0.000000,0.000000,0.000000,0.000000,0.090000,0.000000e+00
25%,7.900000,23.500000,18.100000,17.031875,17.800000,0.000000,5.443200,2.100000e+02
50%,13.700000,36.000000,27.940056,26.700000,25.400100,0.700000,19.051100,7.200000e+02
75%,24.782500,71.125000,44.450100,79.100000,44.800000,13.335013,65.318000,3.613250e+03
max,914.400000,83.800000,9140.000000,8321.056600,9144.000000,1808.483617,185067.585957,6.283065e+06


#### 50 missing titles
#### 1000 missing Titles
#### 2000 missing Names
#### 2000 missing Dates
#### 20000 missing Medium
#### The dimensions, diameter, circumference, height, length, width, depth, weight, duration is normal to have NaNs because it
#### doesn't have to match all criteria depending on the medium.
#### 6000 missing Acquisition dates

In [19]:
artwork.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130262 entries, 0 to 130261
Data columns (total 21 columns):
Artwork ID            130262 non-null object
Title                 130210 non-null object
Artist ID             130262 non-null object
Name                  128802 non-null object
Date                  127954 non-null object
Medium                118343 non-null object
Dimensions            118799 non-null object
Acquisition Date      124799 non-null object
Credit                127192 non-null object
Catalogue             130262 non-null object
Department            130262 non-null object
Classification        130262 non-null object
Object Number         130262 non-null object
Diameter (cm)         1399 non-null float64
Circumference (cm)    10 non-null float64
Height (cm)           111893 non-null float64
Length (cm)           736 non-null float64
Width (cm)            111003 non-null float64
Depth (cm)            11443 non-null float64
Weight (kg)           298 non-null flo

In [20]:
artwork["Artist ID"].str.isnumeric().value_counts()

True     122373
False      7889
Name: Artist ID, dtype: int64

#### Some artworks were done by multiple people, or aren't even referenced back to an Artist
#### Important to note that I might then need to clean up this data to get more accurate answers for Question 2 or 3

In [21]:
artwork.loc[~artwork["Artist ID"].str.isnumeric(), "Artist ID"]

65                          6969, 8134
66                          6956, 6957
76                    7661, 8131, 8180
107                          27, 24452
110                   8102, 6703, 4312
111                   8102, 6703, 4312
112                   8102, 4312, 6703
113                   8102, 6703, 4312
151                   8707, 6951, 6952
154             6956, 7559, 7558, 6957
160                   8213, 6132, 8214
161             8213, 6132, 8214, 8216
162             8215, 6132, 8214, 8216
163             8215, 6132, 8214, 8216
164             8215, 6132, 8214, 8216
178                         6076, 8112
195                         6934, 8103
196                         6934, 8103
238                        8218, 22884
239                        8218, 22884
240                        8218, 22884
241                 29711, 8218, 22884
242             6956, 7558, 7559, 6957
253                          27, 24452
254                   3381, 8157, 8158
266                   338

In [22]:
artwork["Artwork ID"].str.isnumeric().value_counts()

True    130262
Name: Artwork ID, dtype: int64

#### Good thing there's no repeating index

In [23]:
len(artwork["Artwork ID"].unique())

130262

#### We have forks on dates instead of specific years, so the format isn't even consistent

In [24]:
artwork.loc[:20, "Date"]

0        1896
1        1987
2        1903
3        1980
4        1903
5     1976-77
6     1976-77
7     1976-77
8     1976-77
9     1976-77
10    1976-77
11    1976-77
12    1976-77
13    1976-77
14    1976-77
15    1976-77
16    1976-77
17    1976-77
18    1976-77
19    1976-77
20    1976-77
Name: Date, dtype: object

#### The date format is not consistent. Sometimes its trying to fork using the last 2 digits sometimes it actually puts the year

In [25]:
artwork[artwork["Date"].str.match("[0-9]+-[0-9]{4}", na = False)]

,Artwork ID,Title,Artist ID,Name,Date,Medium,Dimensions,Acquisition Date,Credit,Catalogue,...,Classification,Object Number,Diameter (cm),Circumference (cm),Height (cm),Length (cm),Width (cm),Depth (cm),Weight (kg),Duration (s)
789,879,"Generator Project, White Oak, Florida, Design ...",7986,Cedric Price,1977-1978,.a: ink and color ink on paper\n.b: ink and co...,"Each: 7 x 5"" (17.8 x 12.7 cm)",2000-10-31,Gift of The Howard Gilman Foundation,Y,...,Architecture,1263.2000.a-d,NaN,NaN,17.800000,NaN,12.700000,NaN,NaN,NaN
791,881,"Generator Project, White Oak, Florida, Investi...",7986,Cedric Price,1978-1980,.a-c: Ink and ink stamp on paper mounted on paper,"Each: 3 x 5"" (7.6 x 12.7 cm)",None,Gift of The Howard Gilman Foundation,Y,...,Architecture,1265.2000.a-c,NaN,NaN,7.600000,NaN,12.700000,NaN,NaN,NaN
792,882,"Generator Project, White Oak, Florida, Perpsec...",7986,Cedric Price,1977-1978,".a: graphite, crayon and ink on paper mounted ...","Each: 7 x 5"" (17.8 x 12.7 cm)",None,Gift of The Howard Gilman Foundation,Y,...,Architecture,1266.2000.a-d,NaN,NaN,17.800000,NaN,12.700000,NaN,NaN,NaN
797,887,"Generator Project, White Oak, Florida, Sketches",7986,Cedric Price,1978-1980,".a, .e: ink on paper\n.b, .c and .d: ink on pa...",".a-.b: 6 x 4"" (15.2 x 10.2 cm)\n.c-.d: 5 x 3"" ...",None,Gift of The Howard Gilman Foundation,Y,...,Architecture,1271.2000.a-f,NaN,NaN,15.200000,NaN,10.200000,NaN,NaN,NaN
849,940,Furniture Manufacturers Association Headquarte...,7987,Michael Webb,1957-1958,Graphite and ink on tracing paper mounted on b...,"24 x 21 1/4"" (61 x 54 cm)",2000-10-31,Gift of The Howard Gilman Foundation,Y,...,Architecture,1315.2,NaN,NaN,61.000000,NaN,54.000000,NaN,NaN,NaN
882,991,"Chikatsa-Asuka Historical Museum, Minami-Kawac...",7055,Tadao Ando,1989-1993,"Crayon, pencil, colored pencil, ink, charcoal","13 1/4 x 47 1/2"" (33.7 x 120.7 cm)",1993-05-04,Gift of the architect,Y,...,Architecture,141.1993,NaN,NaN,33.655100,NaN,120.650200,NaN,NaN,NaN
892,1002,"Cité de Refuge, Paris, France","3426, 2899","Le Corbusier (Charles-Édouard Jeanneret), Pier...","1929-1933, model 1987",Acrylic,"18 3/4 x 45 x 27"" (47.6 x 114.3 x 68.6 cm)",1987-05-27,Given in honor of Arthur Drexler by Lily Auchi...,Y,...,Architecture,229.1987,NaN,NaN,47.625100,NaN,114.300000,68.580100,NaN,NaN
917,1029,"Glass House, New Canaan, Connecticut",2927,Philip Johnson,1947-1949,"Acrylic, wood, metal, paper, synthetic polymer...","42 1/2 x 55 1/2 x 67"" (108 x 141 x 170.2 cm) S...",1985-11-18,Gift of the architect,Y,...,Architecture,409.1985,NaN,NaN,108.000000,NaN,141.000000,170.200000,NaN,NaN
920,1032,"Vulcania, St. Ours-les-Roches, Auvergne, France",2705,Hans Hollein,1994-2002,Sand-cast basalt,"2 1/2 x 30 1/2 x 17"" (6.4 x 77.5 x 43.2 cm)",1996-09-25,Gift of the architect in honor of Philip Johnson,Y,...,Architecture,411.1996,NaN,NaN,6.400000,NaN,77.500000,43.200000,NaN,NaN
924,1036,"Berlin Museum with the Jewish Museum, Scale mo...",7527,Daniel Libeskind,1989-2001,Wood and paper,"11 1/2 x 119 x 99 1/2"" (29.2 x 302.3 x 252.7 cm)",1996-09-25,Gift of the architect in honor of Philip Johnson,Y,...,Architecture,415.1996,NaN,NaN,29.210058,NaN,302.260605,252.730505,NaN,NaN


#### Some even have parenthesis for the fork

In [26]:
artwork[artwork["Date"].str.match("\([0-9]+-[0-9]{4}\)", na = False)]

,Artwork ID,Title,Artist ID,Name,Date,Medium,Dimensions,Acquisition Date,Credit,Catalogue,...,Classification,Object Number,Diameter (cm),Circumference (cm),Height (cm),Length (cm),Width (cm),Depth (cm),Weight (kg),Duration (s)
31790,33280,Rocks Near the Caves above Château Noir,1053,Paul Cézanne,(1895-1900),Watercolor and pencil on paper,"12 1/2 x 18 3/4"" (31.7 x 47.5 cm)",1934-09-23,Lillie P. Bliss Collection,Y,...,Drawing,21.1934,NaN,NaN,31.700000,NaN,47.500000,NaN,NaN,NaN
32889,34536,"Set design for Der Blaue Cabaret Theatre, Berlin",5821,Pavel Tchelitchew,(1921-1923),Tempera on paper,"7 3/8 x 9 7/8"" (18.6 x 25.0 cm)",1971-03-30,Gift of Mme Alexandra Zaousaileff,Y,...,Drawing,131.1971,NaN,NaN,18.700000,NaN,25.100000,NaN,NaN,NaN
34082,36091,How Briefly - The Tale of Two Meeting,529,Jake Berthot,(1981-1982),Synthetic polymer paint and pencil on paper,"30 1/8 x 22 1/2"" (76.5 x 57.2 cm)",1983-03-15,Acquired with matching funds from Blanchette H...,Y,...,Drawing,300.1983.4,NaN,NaN,76.500000,NaN,57.200000,NaN,NaN,NaN
34083,36092,How Briefly - The Tale of Two Meeting,529,Jake Berthot,(1981-1982),Synthetic polymer paint and pencil on paper,"30 1/8 x 22 3/8"" (76.6 x 56.9 cm)",1983-03-15,Acquired with matching funds from Blanchette H...,Y,...,Drawing,300.1983.5,NaN,NaN,76.600000,NaN,56.900000,NaN,NaN,NaN
34161,36183,Background #23,1641,Dana Duff,(1985-1987),Charcoal on paper,"11 x 14 7/8"" (28 x 37.8 cm)",1989-11-21,Purchase,Y,...,Drawing,308.1989,NaN,NaN,27.900000,NaN,37.800000,NaN,NaN,NaN
35457,37799,Still Life,6177,Maurice de Vlaminck,(1913-1914),Watercolor and gouache on paper,"15 5/8 x 18 7/8"" (39.7 x 47.9 cm)",1949-11-22,Gift of Justin K. Thannhauser,Y,...,Drawing,693.1949,NaN,NaN,39.700000,NaN,47.900000,NaN,NaN,NaN
35747,38203,Portrait of a Woman,4038,Amedeo Modigliani,(1917-1918),Pencil on paper,"17 x 10 3/8"" (43.0 x 26.3 cm)",1979-05-07,Gift of Bella and Sol Fishko,Y,...,Drawing,874.1978,NaN,NaN,43.200000,NaN,26.400000,NaN,NaN,NaN
36186,38708,Untitled,8320,John Morris,(1992-2000),Ink on paper,"10 1/4 x 7"" (26 x 17.8 cm)",2000-12-12,Purchased with funds provided by Sarah-Ann and...,Y,...,Drawing,1611.2,NaN,NaN,26.000000,NaN,17.800000,NaN,NaN,NaN
36187,38709,Untitled,8320,John Morris,(1992-2000),Ink on paper,"10 1/4 x 7"" (26 x 17.8 cm)",2000-12-12,Purchased with funds provided by Sarah-Ann and...,Y,...,Drawing,1612.2,NaN,NaN,26.000000,NaN,17.800000,NaN,NaN,NaN
56472,60093,Three-Sided Elevation on Red,2195,Hermann Glockner,(1971-1977),Tempera pliage,"composition: 28 5/8 x 19 11/16"" (72.7 x 50 cm)...",1979-01-23,Gift of Staatliche Kunstsammlungen Dresden (by...,Y,...,Print,19.1979,NaN,NaN,72.700000,NaN,50.000000,NaN,NaN,NaN


In [27]:
artwork["Medium"].value_counts()

Gelatin silver print                                                                                                                                                                                                                                                                             14103
Lithograph                                                                                                                                                                                                                                                                                        7034
Albumen silver print                                                                                                                                                                                                                                                                              4845
Lithograph, printed in color                                                                                       

In [28]:
artwork["Acquisition Date"].head()

0    1996-04-09
1    1995-01-17
2    1997-01-15
3    1995-01-17
4    1997-01-15
Name: Acquisition Date, dtype: object

In [29]:
artwork["Acquisition Date"].str.match("[0-9]{4}-[0-9]{2}-[0-9]{2}", na = True).value_counts()

True     130255
False         7
Name: Acquisition Date, dtype: int64

#### The acquisition date format isn't consistent either, it usually goes YYYY-MM-DD, but here we can see it only use YYYY-MM
#### Also it could even be DD, we don't know

In [30]:
artwork.loc[~artwork["Acquisition Date"].str.match("[0-9]{4}-[0-9]{2}-[0-9]{2}", na = True), "Acquisition Date"]

130209    2016-10
130210    2016-05
130211    2016-05
130212    2016-10
130213    2016-10
130214    2016-10
130215    2016-05
Name: Acquisition Date, dtype: object

#### One of the acquisition year is from 1216, which makes no sense for the Museum of Modern Art as it was established in 1929

In [31]:
aquiyear = artwork["Acquisition Date"].str.slice(0,4).astype(float)
aquiyear.describe()

count    124799.000000
mean       1983.524572
std          22.593042
min        1216.000000
25%        1965.000000
50%        1983.000000
75%        2006.000000
max        2016.000000
Name: Acquisition Date, dtype: float64

In [32]:
artwork["Credit"].value_counts().head()

The Louis E. Stern Collection                                10927
Purchase                                                      8398
Gift of the artist                                            7191
Abbott-Levy Collection. Partial gift of Shirley C. Burden     4889
The Gilbert and Lila Silverman Fluxus Collection Gift         4603
Name: Credit, dtype: int64

In [33]:
artwork["Catalogue"].value_counts()

Y    71333
N    58929
Name: Catalogue, dtype: int64

In [34]:
artwork["Department"].value_counts()

Prints & Illustrated Books               60128
Photography                              29161
Architecture & Design                    18269
Drawings                                 11027
Painting & Sculpture                      3806
Film                                      3088
Media and Performance Art                 2627
Fluxus Collection                         2135
Architecture & Design - Image Archive       21
Name: Department, dtype: int64

#### Not very clear what they mean by Mies van der Rohe and Frank Lloyd Archive in this column

In [35]:
artwork["Classification"].value_counts()

Print                             30807
Photograph                        29909
Illustrated Book                  26160
Drawing                           11735
Design                            11223
Mies van der Rohe Archive          3331
Architecture                       2947
Video                              2363
Film                               2292
Painting                           2270
Sculpture                          1669
Multiple                           1030
(not assigned)                     1029
Frank Lloyd Wright Archive          785
Periodical                          741
Installation                        596
Work on Paper                       436
Audio                               429
Media                               343
Ephemera                             89
Textile                              33
Performance                          24
Collage                               9
Photography Research/Reference        4
Film (object)                         3


#### I don't know what this is. There seems to be number and codes

In [36]:
artwork["Object Number"]

0            885.1996
1              1.1995
2              1.1997
3              2.1995
4              2.1997
5            3.1995.1
6         3.1995.1-24
7           3.1995.10
8           3.1995.11
9           3.1995.12
10          3.1995.13
11          3.1995.14
12          3.1995.15
13          3.1995.16
14          3.1995.17
15          3.1995.18
16          3.1995.19
17           3.1995.2
18          3.1995.20
19          3.1995.21
20          3.1995.22
21          3.1995.23
22          3.1995.24
23           3.1995.3
24           3.1995.4
25           3.1995.5
26           3.1995.6
27           3.1995.7
28           3.1995.8
29           3.1995.9
             ...     
130232         FC5013
130233         FC5014
130234     FC5015.1-2
130235         FC5016
130236         FC5017
130237         FC5018
130238     FC5019.1-2
130239         FC5020
130240         FC5021
130241         FC5022
130242         FC5023
130243         FC5024
130244         FC5025
130245     FC5026.1-3
130246    

# Question 6

In [37]:
artists.loc[artists["Gender"] == "male", "Gender"] = "Male"
artists["Gender"].value_counts()

Male      9826
Female    2193
Name: Gender, dtype: int64

In [38]:
innermerge = pd.merge(artists, artwork, on='Artist ID', how='inner')

In [39]:
innermergeMen = innermerge[innermerge["Gender"] == "Male"]
innermergeWomen = innermerge[innermerge["Gender"] == "Female"]

#### We can seperate the artworks by gender, we see the MoMA has 6x more art made by men than women

In [40]:
innermergeMen.shape[0]/innermergeWomen.shape[0]

6.174519854258013

In [41]:
innermerge["AcquiYear"] = innermerge["Acquisition Date"].str.slice(0,4).astype(float)

In [42]:
innermerge["AcquiYear"].describe()

count    117243.000000
mean       1982.427045
std          22.448878
min        1216.000000
25%        1964.000000
50%        1980.000000
75%        2005.000000
max        2016.000000
Name: AcquiYear, dtype: float64

#### Cluster it by acquisition dates

In [43]:
EarlyAcquisition = innermerge.loc[innermerge["AcquiYear"] < 1960, :]
MidAcquisition = innermerge.loc[(innermerge["AcquiYear"] >= 1960) & (innermerge["AcquiYear"] < 2000), :]
LateAcquisition = innermerge.loc[innermerge["AcquiYear"] >= 2000, :]

In [44]:
print(EarlyAcquisition.shape)
print(MidAcquisition.shape)
print(LateAcquisition.shape)

(15588, 27)
(63350, 27)
(38305, 27)


#### I can also cluster it based on Date but date has to be extensively cleaned up. Theres quite a lot of missing values and its extremely inconsistent.

#### We can also cluster by classification

In [45]:
innermerge["Classification"].value_counts()

Print                             29786
Photograph                        28400
Illustrated Book                  25326
Drawing                           11509
Design                             9793
Mies van der Rohe Archive          3305
Painting                           2260
Architecture                       2196
Video                              1944
Film                               1882
Sculpture                          1653
Multiple                            961
Frank Lloyd Wright Archive          730
(not assigned)                      723
Installation                        553
Periodical                          444
Media                               283
Audio                               256
Work on Paper                       214
Ephemera                             83
Textile                              32
Performance                          21
Collage                               9
Film (object)                         3
Software                              3


In [46]:
Print = innermerge[innermerge["Classification"] == "Print"]
Photography = innermerge[innermerge["Classification"] == "Photograph"]
Book = innermerge[innermerge["Classification"] == "Illustrated Book"]
Drawing = innermerge[innermerge["Classification"] == "Drawing"]
Paint = innermerge[innermerge["Classification"] == "Painting"]
Sculpture = innermerge[innermerge["Classification"] == "Sculpture"]
#Lets merge the archives together, more research would be needed to see if its appropriate to do so
Archives = innermerge[(innermerge["Classification"] == "Frank Lloyd Wright Archive") |
                      (innermerge["Classification"] == "Mies van der Rohe Archive")]
#We combine all types of media because media could be visual or audio
Media = innermerge[(innermerge["Classification"] == "Video") |
                      (innermerge["Classification"] == "Film") |
                      (innermerge["Classification"] == "Audio") |
                      (innermerge["Classification"] == "Media")]

#They're too small to have their own category so we'll put them as others
not_others = ["Print", "Photograph", "Illustrated Book", "Drawing", "Painting", "Sculpture", "Frank Lloyd Wright Archive",
             "Mies van der Rohe Archive", "Video", "Film", "Audio", "Media"]
Other = innermerge[~(innermerge["Classification"].isin(not_others))]

#### We could classify by department

In [47]:
innermerge["Department"].value_counts()

Prints & Illustrated Books               57748
Photography                              27810
Architecture & Design                    16009
Drawings                                 10865
Painting & Sculpture                      3790
Film                                      2407
Media and Performance Art                 2233
Fluxus Collection                         1493
Architecture & Design - Image Archive       18
Name: Department, dtype: int64

In [48]:
PrintDep = innermerge[innermerge["Department"] == "Prints & Illustrated Books"]
PhotographyDep = innermerge[innermerge["Department"] == "Photography"]
DesignDep = innermerge[innermerge["Department"] == "Architecture & Design"]
DrawingsDep = innermerge[innermerge["Department"] == "Drawings"]
PaintAndSculptureDep = innermerge[innermerge["Department"] == "Painting & Sculpture"]
#Here we can combine Film with Media and Performance Art because they're small enough compared to the other categories
#and similar enough for us to do it
MediaDep = innermerge[(innermerge["Department"] == "Media and Performance Art") | (innermerge["Department"] == "Film") ]
#They're too small to have their own category so we'll put them as others
OtherDep = innermerge[(innermerge["Department"] == "Fluxus Collection") | (innermerge["Department"] == "Architecture & Design - Image Archive")]